# Student Performance Insights : From Cleaning to Clustering to Classification
## Name:Habiba abdullah said hammad

In [1]:
import warnings
warnings.filterwarnings('ignore')

### Libraries used

In [2]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn

### Data Prepration

Ok let's first start with downloading the dataset from UCI ML Repository

In [ ]:
%pip install ucimlrepo -q
from ucimlrepo import fetch_ucirepo


ds = fetch_ucirepo(id=320)
X = ds.data.features
y = ds.data.targets

Note that the server may be down which can cause problems of showing the dataset, however the dataset is already saved as a csv so if the server is down use that

Let's look at the meta data of the dataset 
meta data is data about data so we will have a deeper understanding of the dataset itself

In [ ]:
ds.metadata

This dataset is a *merged* view of the students math scores and porteguse scores
Ok looks like there are 649 students and 30 features. 
it also claims that there is no missing values or even symbols for missing values.
Let's take a look at the variables

In [ ]:
pd.set_option("display.max_colwidth",None)
ds.variables

Ok so most features here are integers except for 
- school 
- sex 
- famsize
- Mjob (Mother's Job)
- Fjob (Father's Job)
- Guardian 


So these would need encoding and given that there are binary features we can one-hot encode them for consistency.
Now let's look at the dataset itself.


In [ ]:
X.head()

In [ ]:
#will need this is in visulaization and EDA
X_original=X.copy()

In [ ]:
y.head()

Ok for reproduciatlity , I will save the dataset as csv files in the Data folder

In [ ]:
X.to_csv("E:\Summer Semester 2\Training\Final Project\Data\student_features.csv",index=False)
y.to_csv("E:\Summer Semester 2\Training\Final Project\Data\student_targets.csv",index=False)

If for any reason this doens't work , the files are avalibale as csv.

In [14]:
X=pd.read_csv("E:\Summer Semester 2\Training\Final Project\Data\student_features.csv")
y=pd.read_csv("E:\Summer Semester 2\Training\Final Project\Data\student_targets.csv")

In [15]:
X.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,4
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,2
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,6
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,0
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,0


In [16]:
y.head()

,G1,G2,G3
0,0,11,11
1,9,11,11
2,12,13,12
3,14,14,14
4,11,13,13


Now that that's setteled, Let's check for duplicates and missing values

In [ ]:
dup_count=X.duplicated().sum()
dup_count

In [ ]:
missing_count=X.isnull().sum()
missing_count

Ok so no duplicates or null values , let's now check for outliers , I will use the IQR method since it is the most simple and when plotted in a box plot it makes it easier to detect outliers while still vieweing the distribution of the features.

This function will check for outliers for all the numeric values 

In [ ]:
def detect_outliers_iqr(df, threshold=1.5):
    outlier_summary = {}

    for col in df.select_dtypes(include=["int64"]).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        lower = Q1 - threshold * IQR
        upper = Q3 + threshold * IQR

        outliers = df[(df[col] < lower) | (df[col] > upper)]
        outlier_summary[col] = {
            "outlier_count": outliers.shape[0],
            "lower_bound": lower,
            "upper_bound": upper,
            "min_value": df[col].min(),
            "max_value": df[col].max()
        }
    return pd.DataFrame(outlier_summary).T.sort_values("outlier_count", ascending=False)


And this function is to plot boxplots

In [ ]:
def plot_outliers_boxplots(df):
    numeric_cols = df.select_dtypes(include=["int64"]).columns
    n = len(numeric_cols)

    rows = (n // 2) + (n % 2)
    plt.figure(figsize=(12, 4*rows))

    for i, col in enumerate(numeric_cols, 1):
        plt.subplot(rows, 2, i)
        sns.boxplot(x=df[col], color="skyblue")
        plt.title(f"Boxplot of {col}")
        plt.xlabel(col)

    plt.tight_layout()
    plt.show()

In [ ]:
outlier_report=detect_outliers_iqr(X)
outlier_report

In [ ]:
plot_outliers_boxplots(X)

Looking at the features , absences at first glance look a little off but after some thinking , yes some students can have be absent for most of the year (Reasons could be Travel) but to not let it affect the model , we can replace the values beyond a threshold with the nearest non-outlier value.

Now for featues like Famrel , FreeTime and Dalc (Daily alcoholic consumption) have some outliers but theya re not significant and are also logical. 

Finally , the failures feature , at first glance , looks like it needs to be dropped right away but actually it is an ordinal Categorical value with values ranging from 0 (no past failures) to 3 (3 or more past failures)

Let's cap absences at 95th percentile for top 5% of values above 30 

In [ ]:
upper=X['absences'].quantile(0.95)
X['absences']=X['absences'].clip(upper=upper)

In [ ]:
Q1 = X['absences'].quantile(0.25)
Q3 = X['absences'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Lower Bound for absences: {lower_bound}")
print(f"Upper Bound for absences: {upper_bound}")

In [ ]:
outliers_lower_after = (X['absences'] < lower_bound).sum()
outliers_upper_after = (X['absences'] > upper_bound).sum()
total_outliers_after = outliers_lower_after + outliers_upper_after

print(f"Lower Outliers after capping: {outliers_lower_after}")
print(f"Upper Outliers after capping: {outliers_upper_after}")
print(f"Total Outliers after capping: {total_outliers_after}")

Now the maximum value is 12 absences. Let's go on to the target features

In [ ]:
outlier_target_report=detect_outliers_iqr(y)
outlier_target_report

In [ ]:
plot_outliers_boxplots(y)

So looking at the target features , there seemes to be no issue since they are withtin the range of 0-20 which is stated in the 'variable_info' in the metadata. Yes there is some outliers but they are logical since students can get a zero or get a below average score therfore the target dataframe stays as is.

Data Quality Report Can be found in the Reports Folder

### Data Transformation

First step in Data Transformation is Encoding 
I wil use one-hot encoding for categorical and binary features

In [ ]:
Categorical_columns=['school','address','famsize','Pstatus','Mjob','Fjob','reason','guardian']
binary_columns=['sex','schoolsup','famsup','paid','activities','nursery','higher','internet','romantic']


First let's handle the binary features

In [ ]:
for col in binary_columns:
    X[col]=X[col].map({"yes":1,"no":0}).fillna(X[col])
    if col=="sex":
        X[col]=X[col].map({"F":0,"M":1})

Now onto the categorical features

In [ ]:
X=pd.get_dummies(X,columns=Categorical_columns,drop_first=True)

In [ ]:
X.head()

Now that the encoding is settled , let's go on to scaling for numeric features. Since I will be using KNN it is important for all the numeric features to be scaled

In [ ]:
from sklearn.preprocessing import StandardScaler
numeric_features=['age','Medu','Fedu','traveltime','studytime','failures','famrel','freetime','goout','Dalc','Walc','health','absences']
scaler=StandardScaler()
X[numeric_features]=scaler.fit_transform(X[numeric_features])
X[numeric_features].head()

Now all the values range from -3 to +3

Now We can go on to feature engineering.
We will do two things 
1. Average of G1-G3 
2. Attendance Proxy
3. Family academic Enviroment
4. Support Index
5. Lifestyle Score
6. Risk Group 

In [ ]:
X["G1"]=y['G1']
X['G2']=y['G2']
X['G3']=y['G3']

Attendence Rate

In [ ]:
X["attendance_rate"] = 1 - (X_original['absences'] / X_original['absences'].max())
X['attendance_rate']

Risk Group (for classification)

In [ ]:
def risk_group(g3):
    if g3<=9: return "High"
    if g3<=14: return "Medium"
    return "Low"
X['risk_group']=y['G3'].apply(risk_group)

Average of G1-G3

In [ ]:
X['avg_grade']=(X['G1']+X['G2']+X['G3'])/3

Now we need to consider leakage 
having G1 and G2 would make it easy for the model to predict G3 and can cause leakage 
so to surpass this we will have 2 dataframes 
one with G1 and G2 
nd one without G1 and G2

In [ ]:
df_with=X.drop(columns=['G3','risk_group'])
df_with

In [ ]:
drop_cols=["G1",'G2','G3','avg_grade','risk_group']
df_without=X.drop(columns=[c for c in drop_cols if c in X.columns])
df_without

In [ ]:
#Classification
y_with_cls=X['risk_group']
y_without_cls=X['risk_group']
#Regression 
y_with_rg=X['G3']
y_without_rg=X['G3']

Ok let's confirm we actually did this step correctly

In [ ]:
df_with.columns.tolist()

In [ ]:
df_without.columns.tolist()

Here are the G3 grades but as categories with the previous function risk_group

In [ ]:
y_with_cls.head()

In [ ]:
y_without_cls.head()

Here are the G3 grades (actual values )

In [ ]:
y_without_rg.head()

In [ ]:
y_with_rg.head()

In [ ]:
"avg_grade" in df_with.columns

In [ ]:
"avg_grade" in df_without.columns

In [ ]:
df_with.shape

In [ ]:
df_without.shape

All of the sanity checks are correct. Now the dataset is ready for EDA and visualization 

### EDA

Now We will try to explore the data even more. For EDA I will use a merged df with the original features and the target variables called df_corr

In [ ]:
df_corr=X_original.copy()
df_corr['G1']=y['G1']
df_corr['G2']=y['G2']
df_corr['G3']=y['G3']
num_features=numeric_features+['G1','G2','G3']

Let's start with descriptive stats table for the  numeric features

In [ ]:
num_summary=(
    df_corr[num_features].describe(percentiles=[0.25,0.5,0.75])
    .T
    .rename(columns={'25%':'q1','50%':'q2','75%':'q3'})
    .round(2)
)
num_summary

And let's not forget the categorical features

In [ ]:
cat_summary=pd.DataFrame({
    'unique_values':df_corr[Categorical_columns].nunique(),
    'most_frequent':df_corr[Categorical_columns].mode().iloc[0],
    'freq_count':df_corr[Categorical_columns].apply(lambda x:x.value_counts().iloc[0]),
                        })
cat_summary['freq_percent']=(cat_summary['freq_count']/len(df_corr)*100).round(2)
cat_summary

Now let's do Correlation analysis
notice that this will be done for the numeric features only 

In [ ]:
corr=df_corr[num_features].corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr,annot=True,fmt=".2f",cmap="coolwarm",cbar=True,square=True)
plt.title("Correlation Heatmap of Numeric Features",fontsize=14)
plt.show()

Ok so looks like G3 has a  strong positivie correlation with G1(0.83) and G2(0.92)
it looks like it has a strong negative correlation failures (-0.39) which means the more failures the lower the value of G3/ 
Suprisingly, Study time has a weak positive correlation and absences has a very weak or no correlation.
So the important predictors for G3 is G1 , G2 and failures.


Now let's do Group Comparisons 
- Study time vs outcomes 
- absences vs outcomes 
- School support vs outcomes 
- failures vs outcomes
- sex vs outcomes 
- lifestyle score vs outcomes

Study time vs outcomes

In [ ]:
plt.figure(figsize=(6,8))
sns.boxplot(x='studytime',y='G3',data=df_corr)
plt.title("Final Grade vs Study Time",fontsize=14)
plt.xlabel("Study time (1=Low , 4=High)")
plt.ylabel("Final Grade")
plt.show()

So there is a slight difference between different study times and final grade  but it isn't that strong.

Absences vs outcomes

In [ ]:
plt.figure(figsize=(6,4))
sns.scatterplot(x='absences',y='G3',data=df_corr,alpha=0.6)
plt.title("Final Grade vs Absences",fontsize=14)
plt.ylabel("Absences")
plt.xlabel("Final Grade")
plt.show()

Looks like most points are clusterd around low absences (0-15) and final grades (5-15), however there is a weak negative correlation and the relationship is not that strong

School Support vs outcomes

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='schoolsup',y='G3',data=df_corr)
plt.title("Final Grade vs School Support",fontsize=14)
plt.xlabel("School Support (Yes/No)")
plt.ylabel("Final Grade")
plt.show()

Students with no support are performing slightly higher (likely because students with support are already at risk)
So when thinking about it , students with suport are performing slighly better but still it is not that significant

Failures vs Outcomes

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='failures',y='G3',data=df_corr)
plt.title("Final Grade vs Past Failures")
plt.xlabel("Past Failures")
plt.ylabel("Final Grade")
plt.show()

Here there is a clear negative correlation as faiulres increase , the final grade decreases 

Sex vs Outcomes

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='sex',y='G3',data=df_corr)
plt.title("Final Grade vs Sex",fontsize=14)
plt.xlabel("Sex (F=Female, M=male)")
plt.ylabel("Final Grade")
plt.show()

In [ ]:
df_corr.groupby('sex')['G3'].mean().round(2)

Looks like female are slightly overperforming but it is still not that significant

Lifestyle score vs outcomes
(Lifestyle score will combine the scores of goout, health and freetime)

In [ ]:
df_corr['Lifestyle_score']=(df_corr['goout'] + df_corr['freetime'] + df_corr['health'])/3
def categorize_lifestyle(score):
    if score <2.5: 
        return 'Low'
    elif score <3.5:
        return 'Medium'
    else:
        return 'High'

df_corr['lifestyle_group']=df_corr['Lifestyle_score'].apply(categorize_lifestyle)
plt.figure(figsize=(6,4))
sns.boxplot(x='lifestyle_group',y='G3',data=df_corr)
plt.title("Final Grade vs Lifestyle group")
plt.show()


Let's calculate the mean to get a good feel about the data 

In [ ]:
df_corr.groupby('lifestyle_group')['G3'].mean().round(2)

so there is not much of a difference between them and looks like they are not strong indicators of the final grade

Hypotheses 
1. Absences are negatively correlated with final grades 
   Evidence: The heatmap shows -0.09
   The scatter plot shows no direct relation 
   Conclusion: Hypothesis Supported but weak effect
2. Students with more past failures have lower final grades
   Evidence: the heatmap shows a correlation of -0.39 
   Students with more than 2 failures have lower grades from the box plot 
   Conclusion: Hypothesis Supported with failures being a meaningful predictor
3. Sex isn't correlated to final grade
   Evidence:The boxplot is not that different 
   Means are not that different 
   Conclusion: Hypothesis Supported
4. Study time is strongly correlated to final grade
    Evidence: After 2 hours , there is not that much of a difference. 
    Conclusion: Hypothesis Rejected (Weak effect)
5. Students with  no school support  outperform students with school support
    Evidence: Although the boxplot of students with no school support has better boxplot but when thinking about it school support is for students who actually need it. 
    Conclusion: Hypothesis Rejected

Before going on to the next module , for a one line summary 
acadmic history is far more infleunetial on final outcomes than behavioral , demographic , or support-related factors

### Visualization

Above in the EDA , I already did the correlation heatmap,scatter plot of absences with interpertation, boxplot of studytime and boxplot of schoolsup ,  but I will put it here for simplicity.

Final Grade vs Absencess

In [ ]:
plt.figure(figsize=(6,4))
sns.scatterplot(x='absences',y='G3',data=df_corr,alpha=0.6)
plt.title("Final Grade vs Absences",fontsize=14)
plt.ylabel("Absences")
plt.xlabel("Final Grade")
plt.show()

Correlation Heatmap

In [ ]:
corr=df_corr[num_features].corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr,annot=True,fmt=".2f",cmap="coolwarm",cbar=True,square=True)
plt.title("Correlation Heatmap of Numeric Features",fontsize=14)
plt.show()

Boxplot of school support vs final grade

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='schoolsup',y='G3',data=df_corr)
plt.title("Final Grade vs School Support",fontsize=14)
plt.xlabel("School Support (Yes/No)")
plt.ylabel("Final Grade")
plt.show()

Histogram of 3+ numeric variables 

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(df_corr['G3'],bins=15,kde=True)
plt.title(f"Distribution of G3",fontsize=14)
plt.xlabel('G3')
plt.ylabel("Frequency")
plt.show()

So Final grade distribution looks rightly skewed and most grades range from 10 to 15

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(df_corr['absences'],bins=15,kde=True)
plt.title(f"Distribution of absences",fontsize=14)
plt.xlabel('absences')
plt.ylabel("Frequency")
plt.show()


Looks like most students don't miss classes as 0 is the most frequent value

In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(df_corr['studytime'],bins=15)
plt.title("Distribution of Study time",fontsize=14)
plt.xlabel("Study Time")
plt.ylabel("Frequency")
plt.show()

So looks like most students study between 2 hours to 2.25 hours 

### Unsupervised Learning (K-Means)

As K-means is an unsupervised clustering algorithm, we will  create a feature set for behavior segmentation (which means academics don't matter here , so we will use the df_without and y_without )

In [ ]:
df_without.columns

Feature set for behavior segmentation

In [ ]:
behavioral_features=['studytime','absences','goout','freetime','famsup','schoolsup']

Elbow Method

In [ ]:
from sklearn.cluster import KMeans
X=df_without[behavioral_features]
inertia=[]
K=range(2,11)
for k in K:
    kmeans=KMeans(n_clusters=k,random_state=43,n_init=10)
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)
plt.plot(K,inertia,'bo-')
plt.xlabel('k (Number of clusters)')
plt.ylabel('Inertia (SSE)')
plt.title('Elbow Method')
plt.show()
    

According to the elbow method, a good number of clusters would be 6 

In [ ]:
from sklearn.metrics import silhouette_score
silhouette=[]
for k in K:
    kmeans=KMeans(n_clusters=k,random_state=42,n_init=10)
    labels=kmeans.fit_predict(X)
    silhouette.append(silhouette_score(X,labels))
plt.plot(K,silhouette,'-bo')
plt.xlabel('K (number of clusters)')
plt.ylabel('Average Silhouette Score')
plt.title('Silhouette Method')
plt.show()

For silhouette method , the best K is 4 
The difference between elbow method and silhouette score is that elbow method favors more clusters since it measures how well points fit within clusters , yet the silhouette score balances cohesion vs sepration which means clusters are best seperated and consistent at the k with the highest silhouette score. We will cluster at k=4 and k=6 and look at the difference between clusters

In [ ]:
results = {}

for k in [4, 6]:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X)   
    
    df_temp = df_without.copy()
    df_temp[f'cluster_{k}'] = labels
    
    cluster_profile = df_temp.groupby(f'cluster_{k}').agg(
        size=(f'cluster_{k}', 'size'),
        avg_studytime=('studytime', 'mean'),
        avg_absences=('absences', 'mean'),
        avg_goout=('goout', 'mean'),
        avg_freetime=('freetime', 'mean'),
        avg_famsup=('famsup', 'mean'),
        avg_schoolsup=('schoolsup', 'mean')
    ).reset_index()
    
    results[k] = cluster_profile

print("K=4 Clusters:\n")
results[4]

let's analyze clusters 
- Cluster 0:low studytime , low absences and low solializing -> Quiet/Average students
- Cluster 1:high absences, moderate goout and low studytime -> The skippers
- Cluster 2:very high study time , low absences, low social-> Diligent Students
- Cluster 3->low studytime , low abences , but very high goout and freetime -> Social/Party Group


In [ ]:
print("\nK=6 Clusters:\n")
results[6]

- Cluster 0: very low studytime , very low absences , very low freetime->Quiet students
- Cluster 1:low studytime,low absences, low goout , moderate freetime-> Quiet Students 2
- Cluster 2: low studytime, low absences, very high goout/freetime -> Social party group 
- Cluster 3:very high studytime , low absences, low goout-> Diligent Students
- Cluster 4 :low studytime , very high absences ,High goout/freetime-> The skippers
- Cluster 5: low study time, very high absences , low goout/freetime-> The skippers 2

Notice how the clusters are repeated when k was 6. ( 2 Groups for skippers and 2 groups for quiet)
K=4 is the better option since each group represents a clear behavioral archetype and are better seperated.

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
labels = kmeans.fit_predict(X)   
    
df_temp = df_without.copy()
df_temp[f'cluster_{4}'] = labels
df_temp['G3']=y_with_rg
    
cluster_profile = df_temp.groupby(f'cluster_4').agg(
        size=(f'cluster_4', 'size'),
        avg_studytime=('studytime', 'mean'),
        avg_absences=('absences', 'mean'),
        avg_goout=('goout', 'mean'),
        avg_freetime=('freetime', 'mean'),
        avg_famsup=('famsup', 'mean'),
        avg_schoolsup=('schoolsup', 'mean'),
        avg_G3=('G3','mean')
    ).reset_index()
    

In [ ]:
from sklearn.decomposition import PCA
kmeans=KMeans(n_clusters=4,random_state=42,n_init=10)
labels=kmeans.fit_predict(X)
pca=PCA(n_components=2,random_state=42)
x_pca=pca.fit_transform(X)
plt.figure(figsize=(8,6))
plt.scatter(x_pca[:,0],x_pca[:,1],c=labels,cmap='tab10',s=40,alpha=0.7)
centers_pca=pca.transform(kmeans.cluster_centers_)
plt.scatter(centers_pca[:,0],centers_pca[:,1],c='red',marker='X',s=200,label='Centroids')
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.title("K-means Clusters (K=4)- PCA reduced")
plt.legend()
plt.show()

Here this is a visualization for cluters. Notice that they do overlap as PCA reduces multi-dimensional behavioral features (6D) into two dimesnions (2D), yet each cluster is meaningfully distinct in terms of their centroids and average behaviors

In [ ]:
g3_comparison = df_temp.groupby('cluster_4')['G3'].mean()

Compare  average G3 across clusters

In [ ]:
plt.figure(figsize=(7,5))
g3_comparison.plot(kind='bar',color='skyblue',edgecolor='black')
plt.title("Average G3 per Cluster (K=4)",fontsize='14')
plt.xlabel("Cluster",fontsize=12)
plt.ylabel("Average G3",fontsize=12)
plt.xticks(rotation=0)
plt.ylim(0,20)
for i, val in enumerate(g3_comparison):
    plt.text(i,val+0.2,f"{val:.2f}",ha='center',fontsize=10)
plt.show()


So looks like Cluster 2 (The diligent students) has the highes average G3 grade and they clearly perform best. 
Morever Cluster 1 (The skippers) have the least performance which links skipping class to poor outcomes.
Finally Clusters 0 and 3 are middle-of-the-road students with cluster 3 being more socially active.

### Supervised Learning

Define Target: it will be a 3-class risk that was already created using the function risk_group

Train at least 3 algorithms 
I will use 
- Random Forest
- Logistic Regressoin 
- Decision Tree


but first we split the data 
for each algorithm there is a pipeline that will be followed 
1. Train the model 
2. Use cross validation and perform basic hyperparameter tuning 
3. Report full metrics
4. Interpret Model 

Finally a table will be created at the end to be able to see the differences.
We also need to make sure that this is done for the two data-leakage variants

Dataset with G1 and G2

In [ ]:
from sklearn.model_selection import train_test_split
#stratify here keeps class balance
X_train,X_test,y_train,y_test=train_test_split(df_with,y_with_cls,test_size=0.2,random_state=42,stratify=y_with_cls)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
df_with.head()

Now that we have 3 models and 2 datasets to train we are going to build a function called evaluate_model which will train the model , evaluate it on test data and returns a dictionary of metrics. 
We will also run GridSearch for all the models to get the best values for the hyper-parameters


In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.model_selection import cross_val_score
def evaluate_model(model,X_train,X_test,y_train,y_test,cv=5):
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    metrics={
        "accuracy":accuracy_score(y_test,y_pred),
        "precision_score_macro":precision_score(y_test,y_pred,average="macro"),
        "recall_score_macro":recall_score(y_test,y_pred,average="macro"),
        "f1_score_macro":f1_score(y_test,y_pred,average="macro"),
        "cv_score_macro":np.mean(cross_val_score(model,X_train,y_train,cv=cv,scoring="f1_macro"))
    }
    return metrics

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

tuning_results = {"df_with": {}, "df_without": {}}
results = {"df_with": {}, "df_without": {}}

datasets = {
    "df_with": (df_with, y_with_cls),
    "df_without": (df_without, y_without_cls)
}

param_grids = { 
    "LogisticRegression": {"C": [0.01, 0.1, 1, 10, 100]}, 
    "RandomForest": { 
        "n_estimators": [100, 200, 500], 
        "max_depth": [None, 10, 20], 
        "min_samples_split": [2, 5, 10] 
    }, 
   "DecisionTree": {
        "max_depth": [None, 5, 10, 20],
        "min_samples_split": [2, 5, 10],
        "criterion": ["gini", "entropy"]
    }
}

models = {  
    "LogisticRegression": LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=1000, random_state=42),  
    "RandomForest": RandomForestClassifier(random_state=42),  
    "DecisionTree": DecisionTreeClassifier(random_state=42)  
}

for name, (X, y) in datasets.items():  
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    for model_name, model in models.items(): 
        print(f"Running {model_name} on {name} ...") 
        
        grid = GridSearchCV(
            model, param_grids[model_name], 
            cv=5, scoring="f1_macro", n_jobs=-1
        )
        
        # Train GridSearch
        grid.fit(X_train, y_train)
        
        # Store tuning results
        tuning_results[name][model_name] = {
            "best_params": grid.best_params_,
            "best_cv_f1_macro": grid.best_score_
        }
        
        
        best_model = grid.best_estimator_
        metrics = evaluate_model(best_model, X_train, X_test, y_train, y_test)
        results[name][model_name] = metrics


In [ ]:
results

## Model Evaluation and Comparison

In [ ]:
tuning_rows = []
for dataset, models in tuning_results.items():
    for model_name, vals in models.items():
        tuning_rows.append({
            "Dataset": dataset,
            "Model": model_name,
            "Best Params": vals["best_params"],
            "Best CV F1": vals["best_cv_f1_macro"]
        })
tuning_df = pd.DataFrame(tuning_rows)

result_rows = []
for dataset, models in results.items():
    for model_name, vals in models.items():
        row = {"Dataset": dataset, "Model": model_name}
        row.update(vals)  
        result_rows.append(row)
results_df = pd.DataFrame(result_rows)

final_df = pd.merge(tuning_df, results_df, on=["Dataset", "Model"])


final_df = final_df[
    ["Dataset", "Model", "Best Params", "Best CV F1",
     "accuracy", "precision_score_macro", "recall_score_macro",
     "f1_score_macro", "cv_score_macro"]
]


for col in ["Best CV F1", "accuracy", "precision_score_macro", 
            "recall_score_macro", "f1_score_macro", "cv_score_macro"]:
    final_df[col] = final_df[col].round(3)

final_df


With leakage all models perform well with logistic regression achieving the highest precision. Random Forest also outperformes others in CV F1 while DecisionTree is more interpretable.
Yet without leakage the performance drops significantly. Logisitic regression has the better F1 score compared to others , while RandomForest shows high precision but low recall which makes it clear that it predictes fewer positives correctly 
So final observation is that models overfit when leakage exist and removal of leakage allows more generalization

Now we can look for interpretiability 
for logistic regression it would be the coefficients where large  positive coefficents show features strong associated with a specific risk class and large negative coeeficeints reduces the likelihood of that class

In [ ]:
def get_top_coeffs(X, y, best_params, dataset_name, top_n=10):
    logreg = LogisticRegression(
        **best_params,
        multi_class="multinomial",
        solver="lbfgs",
        max_iter=1000,
        random_state=42
    )
    logreg.fit(X, y)

    coeffs = pd.DataFrame(
        logreg.coef_.T,
        index=X.columns,
        columns=[f"class_{cls}" for cls in logreg.classes_]
    )

    coeffs_sorted = coeffs.sort_values(by=coeffs.columns[0], ascending=False).head(top_n)
    coeffs_sorted["dataset"] = dataset_name
    return coeffs_sorted

top_with = get_top_coeffs(df_with, y_with_cls, tuning_results["df_with"]["LogisticRegression"]["best_params"], "df_with")
top_without = get_top_coeffs(df_without, y_without_cls, tuning_results["df_without"]["LogisticRegression"]["best_params"], "df_without")

comparison = pd.concat([top_with, top_without])

comparison

In df_with, G1 and G2 strongly and positively influence the probability of being in the High-risk class. The coefficients for these features are much larger than others, suggesting data leakage, as these features dominate the model.
In contrast, for df_without, the coefficients are smaller and more balanced, reflecting more realistic predictors. Here, features like school_MS and schoolsup positively influence High risk, indicating that non-grade-related factors are driving the predictions.

In [ ]:
comparison_reset = comparison.reset_index().rename(columns={"index": "Feature"})

fig, axes = plt.subplots(1, 3, figsize=(20,6), sharey=True)

for i, cls in enumerate(["class_High", "class_Low", "class_Medium"]):
    plot_data = comparison_reset[["Feature", cls, "dataset"]].rename(columns={cls: "Coefficient"})
    
    sns.barplot(
        data=plot_data,
        x="Feature",
        y="Coefficient",
        hue="dataset",
        palette="Set2",
        ax=axes[i]
    )
    
    axes[i].set_title(f"LogReg Coefficients for {cls}", fontsize=14)
    axes[i].tick_params(axis="x", rotation=45)

plt.suptitle("Logistic Regression Coefficients Comparison (df_with vs df_without)", fontsize=16, y=1.05)
plt.tight_layout()
plt.show()


Random Forest would have feature importance where features with highest importance are the main drivers of prediction

In [ ]:
rf_with=RandomForestClassifier(**tuning_results["df_with"]["RandomForest"]["best_params"],random_state=42)
rf_with.fit(df_with,y_with_cls)
importances_with=pd.DataFrame({
    "feature":df_with.columns,
    "importance":rf_with.feature_importances_
}).sort_values("importance",ascending=False)
rf_without=RandomForestClassifier(**tuning_results["df_without"]["RandomForest"]["best_params"],random_state=42)
rf_without.fit(df_without,y_without_cls)
importances_without=pd.DataFrame({
    "feature":df_without.columns,
    "importance":rf_without.feature_importances_
}).sort_values("importance",ascending=False)
importances_with["dataset"] = "df_with"
importances_without["dataset"] = "df_without"


rf_importances = pd.concat([importances_with, importances_without])


top_n = 10
top_features = (
    rf_importances.groupby("feature")["importance"]
    .max()
    .sort_values(ascending=False)
    .head(top_n)
    .index
)
plot_data = rf_importances[rf_importances["feature"].isin(top_features)]


plt.figure(figsize=(12,6))
sns.barplot(
    data=plot_data,
    x="feature",
    y="importance",
    hue="dataset",
    palette="Set2"
)

plt.title("Top Random Forest Feature Importances (df_with vs df_without)", fontsize=14)
plt.xticks(rotation=45, ha="right")
plt.ylabel("Feature Importance")
plt.xlabel("Feature")
plt.legend(title="Dataset")
plt.tight_layout()
plt.show()

The Random Forest feature importance plot shows that in df_with, features like avg_grade and G2 dominate the model, confirming the presence of data leakage, as these grade-related features almost fully determine the prediction.
In contrast, for df_without, the importance is more evenly distributed among features like failures, age, health, and famrel, indicating that the model is now relying on more realistic and meaningful predictors. This demonstrates how removing leakage changes the model’s behavior and improves interpretability.

Now for Decision Tree we can try to visualize the best decision tree created

In [ ]:
from sklearn import tree

best_dt_with = DecisionTreeClassifier(random_state=42, **tuning_results["df_with"]["DecisionTree"]["best_params"])
best_dt_with.fit(df_with, y_with_cls)

best_dt_without = DecisionTreeClassifier(random_state=42, **tuning_results["df_without"]["DecisionTree"]["best_params"])
best_dt_without.fit(df_without, y_without_cls)

fig, axes = plt.subplots(1, 2, figsize=(24, 8)) 


tree.plot_tree(
    best_dt_with,
    feature_names=df_with.columns,
    class_names=[str(cls) for cls in best_dt_with.classes_],
    filled=True,
    max_depth=3,          
    fontsize=10,          
    ax=axes[0]
)
axes[0].set_title("Decision Tree (df_with)")


tree.plot_tree(
    best_dt_without,
    feature_names=df_without.columns,
    class_names=[str(cls) for cls in best_dt_without.classes_],
    filled=True,
    max_depth=3,
    fontsize=10,
    ax=axes[1]
)
axes[1].set_title("Decision Tree (df_without)")

plt.tight_layout()
plt.show()

For the left decision tree , the average grade dominates the top node which  suggest that G1 and G2 are strong predictors, yet on the right the top node is failures and relies more on realistic student information. that concluds that df_with suffers from data leakahge.

### Storytelling and Recommendations